In [ ]:
import ipywidgets as w
import nglview as nv

In [ ]:
txt = w.Text(
    value='lorem ipsum',
    disabled=False,
    tooltip='Blabla blablabla.',
    layout=w.Layout(width='90%'))

start = w.Button(
    description='Run',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)

upload = w.FileUpload(accept='pdb',multiple=False)
name = w.Label()

In [ ]:
pdbview = nv.NGLWidget()
cartoon = w.Checkbox(description='Cartoon',value=True)
licorice = w.Checkbox(description='Licorice',value=True)

def on_checkbox_click(e):
    pdbview.clear()
    if cartoon.value:
        pdbview.add_cartoon()
    if licorice.value:
        pdbview.add_licorice()
            

def on_upload_pdb(c):
#    print(upload.value)
    with open('mol.pdb','wb') as f:
        f.write(list(upload.value.values())[0]['content'])
    pdbview.add_component('mol.pdb')
    pdbview.add_cartoon()
    pdbview.add_licorice()
    name.value = list(upload.value.keys())[0]
    
upload.observe(on_upload_pdb,'_counter')
cartoon.observe(on_checkbox_click,names='value')
licorice.observe(on_checkbox_click,names='value')

In [ ]:
import random

def set_random(event):
    txt.value = str(random.randint(0,100)) + " " + str(upload.value[0])
    
start.on_click(set_random)
main = w.VBox([upload, name, pdbview, cartoon, licorice, start])
display(main)

In [ ]:
!ls